In [1]:
!nvidia-smi

Fri Jan  8 04:42:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.89       Driver Version: 460.89       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208... WDDM  | 00000000:03:00.0  On |                  N/A |
|  0%   33C    P8    20W / 300W |   1066MiB / 11264MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#Imports
import transformers
from transformers import XLNetTokenizer, XLNetModel, AdamW, get_linear_schedule_with_warmup
import torch
import re
from transformers import AdamW
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from pylab import rcParams

from torch import nn, optim
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import XLNetTokenizer, XLNetModel, XLNetForSequenceClassification
from torch.cuda.amp import GradScaler, autocast

In [4]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from nlp import load_dataset
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
import datasets
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [5]:
#Set random seet and define device
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [132]:
#Read in dataset
df = pd.read_csv(r'E:\Datasets\Tweet Sentiment Analysis\training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None)

In [133]:
#Drop unneeded columns
df = df.drop([1,2,3,4], axis=1)

In [6]:
def clean_text(text):
    #text = re.sub(r"[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9@#-]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    return text

In [ ]:
#Clean the tweets
df[5] = df[5].apply(clean_text)

In [134]:
#Fix up sentiment labels
def sentiment2label(sentiment):
    if sentiment == 4:
        return 1
    else :
        return 0

df[0] = df[0].apply(sentiment2label)

In [135]:
#Rename columns
df.columns = ['sentiment', 'tweet']

In [136]:
#Split data in to train, test, and val
df_train, df_test = train_test_split(df, test_size=0.2, random_state=101)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=101)

In [137]:
#Save dataframes to csv
df_train.to_csv(r'E:\Datasets\Tweet Sentiment Analysis\train.csv')
df_test.to_csv(r'E:\Datasets\Tweet Sentiment Analysis\test.csv')
df_val.to_csv(r'E:\Datasets\Tweet Sentiment Analysis\val.csv')

In [7]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [8]:
#Initialize tokenizer
PRE_TRAINED_MODEL_NAME = r'E:\Models\bert-large-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [9]:
#Set special token variables
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [10]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [11]:
#Set max input length
max_input_length = 32

In [11]:
#Define tokenize_and_cut function which takes in text and outputs tokens truncated for the max length
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [12]:
#Define data fields
from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [18]:
#Define torchtext datasets
from torchtext import data
from torchtext import datasets

train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = 'E:\\Datasets\\Tweet Sentiment Analysis\\',
                                        train = 'train.csv',
                                        validation = 'val.csv',
                                        test = 'test.csv',
                                        format = 'csv',
                                        fields = [(None, None), ('l', LABEL), ('t', TEXT)],
                                        skip_header = True
)

In [19]:
LABEL.build_vocab(train_data)

print(LABEL.vocab.stoi)

defaultdict(None, {'0': 0, '1': 1})


In [141]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 1280000
Number of validation examples: 160000
Number of testing examples: 160000


In [17]:
print(vars(train_data.examples[1]))

{'l': '1', 't': [1030, 6583, 12171, 27172, 4283, 2005, 3582, 2860, 2860, 2860, 2860, 2860, 2860, 2860, 2860]}


In [18]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[4])['t'])

print(tokens)

['i', 'wanna', 'go', 'act', 'my', 'shoe', 'size', 'an', 'have', 'a', 'water', 'fight']


In [20]:
#Set the batch size and define the device and data iterators
BATCH_SIZE = 1024

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort=False)


In [14]:
#Initialize Bert
bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [15]:
#Define BERTGRUSentiment for regression
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [22]:
#Define hyperparameters
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [23]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [24]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [25]:
#Define binary_accuracy function
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [26]:
#Define train function
def train(model, iterator, optimizer, criterion, scaler):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()

        with autocast():
            predictions = model(batch.t).squeeze(1)
            
            loss = criterion(predictions, batch.l)
            
            acc = binary_accuracy(predictions, batch.l)
        
        scaler.scale(loss).backward()
        
        scaler.step(optimizer)
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        scaler.update()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [27]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            
            with autocast():

                predictions = model(batch.t).squeeze(1)
                
                loss = criterion(predictions, batch.l)
                
                acc = binary_accuracy(predictions, batch.l)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [28]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [49]:
#Run the training loop
#Use accuracy as metric
#Baseline is 50% since there are an equal amount of positive and negative tweets
N_EPOCHS = 50
best_valid_loss = float('inf')
scaler = GradScaler()

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, scaler)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'E:/Datasets/Tweet Sentiment Analysis/models/bert_model2')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 20m 23s
	Train Loss: 0.304 | Train Acc: 86.79%
	 Val. Loss: 0.366 |  Val. Acc: 84.22%
Epoch: 02 | Epoch Time: 19m 26s
	Train Loss: 0.301 | Train Acc: 86.90%
	 Val. Loss: 0.370 |  Val. Acc: 84.20%
Epoch: 03 | Epoch Time: 19m 4s
	Train Loss: 0.299 | Train Acc: 87.04%
	 Val. Loss: 0.372 |  Val. Acc: 84.35%
Epoch: 04 | Epoch Time: 19m 16s
	Train Loss: 0.297 | Train Acc: 87.16%
	 Val. Loss: 0.372 |  Val. Acc: 84.34%
Epoch: 05 | Epoch Time: 19m 17s
	Train Loss: 0.295 | Train Acc: 87.23%
	 Val. Loss: 0.366 |  Val. Acc: 84.30%
Epoch: 06 | Epoch Time: 19m 9s
	Train Loss: 0.293 | Train Acc: 87.34%
	 Val. Loss: 0.371 |  Val. Acc: 84.10%
Epoch: 07 | Epoch Time: 18m 59s
	Train Loss: 0.291 | Train Acc: 87.45%
	 Val. Loss: 0.371 |  Val. Acc: 84.21%
Epoch: 08 | Epoch Time: 19m 0s
	Train Loss: 0.290 | Train Acc: 87.51%
	 Val. Loss: 0.377 |  Val. Acc: 84.16%


KeyboardInterrupt: 

In [30]:
#Load saved model - max 85% accuracy both test and val sets
model.load_state_dict(torch.load('E:/Datasets/Tweet Sentiment Analysis/models/bert_model'))

<All keys matched successfully>

In [31]:
#Calculate test loss and accuracy
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

NameError: name 'test_iterator' is not defined

In [32]:
#Define predict_sentiment function that takes the trained model, tokenizer, and the sentence to analyze
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    #sentence = clean_text(sentence)
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    positive = round(prediction.item() * 100, 2)
    negative = round(100 - positive, 2)
    print(f'Positive: {positive}%')
    print(f'Negative: {negative}%')
    print('-'*25)
    if positive >= 55:
        print('This tweet is positive!')
    elif positive <= 45:
        print('This tweet is negative!')
    else:
        print('This tweet is neutral.')

    print('-'*25)
    print(f'Text = {sentence}')

In [33]:
tweet = """We must end all federal executions and abolish the death penalty. In a world of incredible violence, the state should not be involved in premeditated murder.
"""

In [34]:
predict_sentiment(model, tokenizer, tweet)

Positive: 2.62%
Negative: 97.38%
-------------------------
This tweet is negative!
-------------------------
Text = We must end all federal executions and abolish the death penalty. In a world of incredible violence, the state should not be involved in premeditated murder.



In [70]:
#Load train and test dataframes for tree and linear models
df = pd.read_csv(r'E:\Datasets\Tweet Sentiment Analysis\train.csv').drop('Unnamed: 0', axis=1)
df_test = pd.read_csv(r'E:\Datasets\Tweet Sentiment Analysis\test.csv').drop('Unnamed: 0', axis=1)

In [71]:
#Feature emgineering and then split df in to X feature matrix and y target vector
df['characters'] = df['tweet'].apply(lambda x : len(x))
df['words'] = df['tweet'].apply(lambda x : len(x.split(' ')))
X_train = df[['characters', 'words']]
y_train = df['sentiment']

df_test['characters'] = df['tweet'].apply(lambda x : len(x))
df_test['words'] = df['tweet'].apply(lambda x : len(x.split(' ')))
X_test = df_test[['characters', 'words']]
y_test = df_test['sentiment']

In [74]:
#Initiate and train random forest classifier
from sklearn.ensemble import RandomForestClassifier
modelrf = RandomForestClassifier(criterion='gini', random_state=98, max_depth=28, max_features='auto', n_jobs=-1, n_estimators=200, max_samples=.8)
modelrf.fit(X_train, y_train)

RandomForestClassifier(max_depth=28, max_samples=0.8, n_estimators=200,
                       n_jobs=-1, random_state=98)

In [75]:
#Score random forest classifier
modelrf.score(X_test, y_test)

0.4990375

In [76]:
#Initiate and train logistic regression
from sklearn.linear_model import LogisticRegressionCV
modellr = LogisticRegressionCV()
modellr.fit(X_train, y_train)

LogisticRegressionCV()

In [77]:
#Score logistic regression
modellr.score(X_test, y_test)

0.49908125